## SAMPLING MULTIPLE RASTERS USING POINTS (gpkg)

In [1]:
# Import required libraries
import geopandas as gpd
import rasterio
from rasterio.sample import sample_gen
from pathlib import Path
import numpy as np

def read_points(gpkg_path):
    """
    Read point GeoPackage and return GeoDataFrame
    
    Parameters:
    gpkg_path (str): Path to input point GeoPackage
    
    Returns:
    GeoDataFrame: Points with their attributes
    """
    try:
        points_gdf = gpd.read_file(gpkg_path)
        
        # Verify it's a point geometry
        if not all(points_gdf.geometry.type == 'Point'):
            raise ValueError("Input GeoPackage must contain only Point geometries")
            
        return points_gdf
    except Exception as e:
        print(f"Error reading GeoPackage: {e}")
        return None

def extract_raster_values(points_gdf, raster_path):
    """
    Extract raster values for each point location, ignoring negative values
    
    Parameters:
    points_gdf (GeoDataFrame): Points to sample
    raster_path (str): Path to raster file
    
    Returns:
    list: Extracted values for each point, with None for negative values
    """
    try:
        with rasterio.open(raster_path) as src:
            # Get coordinates from points
            xy = [(point.x, point.y) for point in points_gdf.geometry]
            
            # Sample raster values
            values = list(sample_gen(src, xy))
            
            # Convert negative values to None and round to integers
            processed_values = [round(val[0]) if val[0] >= 0 else None for val in values]
            
            return processed_values
    except Exception as e:
        print(f"Error extracting values from {raster_path}: {e}")
        return None

def update_geopackage(gpkg_path, raster_folder, prefix):
    """
    Update input GeoPackage with raster values as new columns
    
    Parameters:
    gpkg_path (str): Path to input point GeoPackage
    raster_folder (str): Path to folder containing TIF files
    prefix (str): Prefix for output column names
    """
    # Read points
    points_gdf = read_points(gpkg_path)
    if points_gdf is None:
        return
    
    # Find all TIF files in folder
    raster_files = list(Path(raster_folder).glob('*.tif'))
    
    if not raster_files:
        print(f"No TIF files found in {raster_folder}")
        return
    
    # Process each raster
    for raster_path in raster_files:
        # Get raster name without extension and create column name with prefix
        raster_name = f"{prefix}_{raster_path.stem}"
        
        print(f"Processing {raster_path.name}...")
        
        # Extract values
        values = extract_raster_values(points_gdf, str(raster_path))
        
        if values is not None:
            # Add values as new column with integer type
            points_gdf[raster_name] = values
            points_gdf[raster_name] = points_gdf[raster_name].astype('Int32')  # Using nullable integer type
            
            # Report number of negative values that were ignored
            num_negative = sum(1 for v in values if v is None)
            if num_negative > 0:
                print(f"Ignored {num_negative} negative values in {raster_path.name}")
    
    # Save updated GeoPackage
    try:
        points_gdf.to_file(gpkg_path, driver='GPKG')
        print(f"Successfully updated {gpkg_path} with new columns")
    except Exception as e:
        print(f"Error saving updated GeoPackage: {e}")

# Example usage
if __name__ == "__main__":
    # Get user inputs
    points_gpkg = input("Enter path to point GeoPackage (.gpkg): ")
    raster_folder = input("Enter path to raster folder: ")
    prefix = input("Enter prefix for output columns: ")
    
    # Update GeoPackage with raster values
    update_geopackage(points_gpkg, raster_folder, prefix)

Enter path to point GeoPackage (.gpkg):  X:\Work\Geodata\CCDR\FJI\Exposure\FJ_Crops2050.gpkg
Enter path to raster folder:  X:\Work\WB\Risk_tools\script\data\HZD\FJI\COASTAL_UNDEFENDED\2050\SSP5_8.5
Enter prefix for output columns:  CU


C:\Users\matti\anaconda3\envs\rdl-tools\lib\site-packages\pyogrio\geopandas.py:261: UserWarning: More than one layer found in 'FJ_Crops2050.gpkg': 'FJ_Crops2050_SSP2' (default), 'FJ_Crops2050_SSP5'. Specify layer parameter to avoid this warning.
  result = read_func(


Processing 1in10.tif...
Ignored 3460 negative values in 1in10.tif
Processing 1in100.tif...
Ignored 3460 negative values in 1in100.tif
Processing 1in1000.tif...
Ignored 3460 negative values in 1in1000.tif
Processing 1in20.tif...
Ignored 3460 negative values in 1in20.tif
Processing 1in200.tif...
Ignored 3460 negative values in 1in200.tif
Processing 1in5.tif...
Ignored 3460 negative values in 1in5.tif
Processing 1in50.tif...
Ignored 3460 negative values in 1in50.tif
Processing 1in500.tif...
Ignored 3460 negative values in 1in500.tif
Successfully updated X:\Work\Geodata\CCDR\FJI\Exposure\FJ_Crops2050.gpkg with new columns
